In [0]:
# !unzip -uq "/content/drive/My Drive/roi.zip" -d "/content/drive/My Drive/roi_sample"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

In [2]:
path_plus='roi_sample/E01' # 무표정
path_zero='roi_sample/E02' # 웃음
path_minus='roi_sample/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:2880
plus 경로에 저장된 파일의 개수:2880
minus 경로에 저장된 파일의 개수:2880


In [3]:
xsize=100
ysize=100

In [4]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [5]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
print(x.shape)

(8640, 100, 100, 3)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/

In [9]:
print(x_test.shape)
print(y_test.shape)

(864, 100, 100, 3)
(864,)


In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications.vgg16 import VGG16 

input_shape = (100, 100, 3)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable=False

x = Flatten()(base_model.output)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()


58892288/58889256 [==============================] - 8s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
___________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
model.fit(x_train, y_train, validation_split=0.2, 
          epochs=30, batch_size=100, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)


Train on 6220 samples, validate on 1556 samples
Epoch 1/30
6220/6220 [==============================] - 29s 5ms/step - loss: 3.0112 - accuracy: 0.3373 - val_loss: 1.0990 - val_accuracy: 0.3239
Epoch 2/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.1091 - accuracy: 0.3379 - val_loss: 1.1054 - val_accuracy: 0.3181
Epoch 3/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.1053 - accuracy: 0.3418 - val_loss: 1.1090 - val_accuracy: 0.3130
Epoch 4/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.0993 - accuracy: 0.3415 - val_loss: 1.0982 - val_accuracy: 0.3573
Epoch 5/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.1005 - accuracy: 0.3370 - val_loss: 1.0980 - val_accuracy: 0.3503
Epoch 6/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.0998 - accuracy: 0.3238 - val_loss: 1.0984 - val_accuracy: 0.3528
Epoch 7/30
6220/6220 [==============================] - 20s 3ms/step - loss: 1.09

In [14]:
model.save('save/InceptionV3_freezing(full).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [0]:
test_path = "test_set" # 기쁨만 모인 test set
test_path2 = "test_set2" # 무표정만 모인 test set 

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [0]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
0
1
2
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1


In [0]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [0]:
# 예측(test set 기쁨)
co=0
for i in range(100):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n',co/100.)


2111111111111111101111111111111111111111111212111111111111111111111211111111101201011111111111111121
 0.9


In [0]:
# 예측(test set 무표정)
co=0
for i in range(100):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n',co/100.)

1101110112110111110111201121111111111111011200011111122100100111021001101111011011210122112111121111
 0.2
